In [1]:
import spacy
nlp = spacy.load("en_core_web_md")

In [22]:
#importing the libraries

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity 
import ml_metrics as metrics

In [3]:
#importing the data of the cleanned genre and plot
df=pd.read_csv('wiki_movie_plots_deduped_cleaned_genre_and_plot.csv')
df

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Genre_updated,Wiki Page,Plot,Plot_cleanned
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,NaN,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr...",bartender working saloon serving drink custome...
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,NaN,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov...",moon painted smiling face hang park night youn...
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,NaN,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed...",film minute long composed two shot first girl ...
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,NaN,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...,lasting second consisting two shot first shot ...
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,NaN,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...,earliest known adaptation classic fairytale fi...
...,...,...,...,...,...,...,...,...,...,...
42530,2014,The Water Diviner,Turkish,Director: Russell Crowe,Director: Russell Crowe\r\nCast: Russell Crowe...,unknown,NaN,https://en.wikipedia.org/wiki/The_Water_Diviner,"The film begins in 1919, just after World War ...",film begin wxyz world war ended centre around ...
42531,2017,Çalgı Çengi İkimiz,Turkish,Selçuk Aydemir,"Ahmet Kural, Murat Cemcir",comedy,comedy,https://en.wikipedia.org/wiki/%C3%87alg%C4%B1_...,"Two musicians, Salih and Gürkan, described the...",two musician salih gürkan described adventure ...
42532,2017,Olanlar Oldu,Turkish,Hakan Algül,"Ata Demirer, Tuvana Türkay, Ülkü Duru",comedy,comedy,https://en.wikipedia.org/wiki/Olanlar_Oldu,"Zafer, a sailor living with his mother Döndü i...",zafer sailor living mother döndü coastal villa...
42533,2017,Non-Transferable,Turkish,Brendan Bradley,"YouTubers Shanna Malcolm, Shira Lazar, Sara Fl...",romantic comedy,comedy,https://en.wikipedia.org/wiki/Non-Transferable...,The film centres around a young woman named Am...,film centre around young woman named amy tyler...


In [4]:
#importing the data of most frequent words/common words in the plot column for each genre
data=pd.read_csv("most_frequent_words_100.csv")
data

,Genre,Words,Counts
0,western,town,1144
1,western,men,940
2,western,take,683
3,western,back,672
4,western,one,671
...,...,...,...
2795,anime,night,51
2796,anime,arrive,51
2797,anime,attacked,51
2798,anime,war,51


In [5]:
#making a dict for the 
dict_words={i:(data[data['Genre']==i]['Words']).to_list() for i in data['Genre'].unique()}
#dict_words

In [6]:
def reduce_words(x):
    return ' '.join([i for i in x['Plot_cleanned'].split() if i in  dict_words[x['Genre_updated']]])
     

In [7]:
%%time
Train=df[~pd.isnull(df['Genre_updated'])]
Test=df[pd.isnull(df['Genre_updated'])]


Train['Plot_trimmed']=Train[['Genre_updated','Plot_cleanned']].apply(lambda x: reduce_words(x),axis=1)


Wall time: 1min 15s


<timed exec>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [8]:
%%time

X_train=Train['Plot_trimmed']
y_train=Train['Genre_updated']


X_train, X_val, y_train, y_val = train_test_split(X_train,y_train,test_size=0.3,random_state=123,stratify=y_train)


X_test=Test['Plot_cleanned']
y_test=Test['Genre_updated']

Wall time: 28.9 ms


In [9]:
%%time
Train['plot_tokenized']=Train['Plot_trimmed'].apply(nlp)
Train['plot_sent_vectors'] = Train['plot_tokenized'].apply(lambda sent: sent.vector)
Train['plot_sent_vectors'] = Train['plot_sent_vectors'].apply(lambda x:x.tolist())
Train

<timed exec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Wall time: 4min 54s


<timed exec>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Genre_updated,Wiki Page,Plot,Plot_cleanned,Plot_trimmed,plot_tokenized,plot_sent_vectors
6,1903,The Great Train Robbery,American,Edwin S. Porter,NaN,western,western,https://en.wikipedia.org/wiki/The_Great_Train_...,The film opens with two bandits breaking into ...,film open two bandit breaking railroad telegra...,two train order train two kill kill train trai...,"(two, train, order, train, two, kill, kill, tr...","[-0.03173771873116493, -0.03446159139275551, -..."
7,1904,The Suburbanite,American,Wallace McCutcheon,NaN,comedy,comedy,https://en.wikipedia.org/wiki/The_Suburbanite,The film is about a family who move to the sub...,film family move suburb hoping quiet life thin...,film family life start go wife get start,"(film, family, life, start, go, wife, get, start)","[0.06740987300872803, 0.1805768758058548, -0.1..."
10,1906,Dream of a Rarebit Fiend,American,Wallace McCutcheon and Edwin S. Porter,NaN,short,short,https://en.wikipedia.org/wiki/Dream_of_a_Rareb...,The Rarebit Fiend gorges on Welsh rarebit at a...,rarebit fiend gorge welsh rarebit restaurant l...,begin get start try onto around man help get h...,"(begin, get, start, try, onto, around, man, he...","[0.10471398383378983, 0.09727063775062561, -0...."
11,1906,From Leadville to Aspen: A Hold-Up in the Rockies,American,Francis J. Marion and Wallace McCutcheon,NaN,short action/crime western,action,https://en.wikipedia.org/wiki/From_Leadville_t...,The film features a train traveling through th...,film feature train traveling rockies hold crea...,film two make along later,"(film, two, make, along, later)","[0.1537410020828247, -0.14758041501045227, -0...."
12,1906,From Leadville to Aspen: A Hold-Up in the Rockies,American,Francis J. Marion and Wallace McCutcheon,NaN,short action/crime western,western,https://en.wikipedia.org/wiki/From_Leadville_t...,The film features a train traveling through th...,film feature train traveling rockies hold crea...,train two make later horse,"(train, two, make, later, horse)","[0.0882359966635704, -0.030830401927232742, -0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
42526,2013,Particle (film),Turkish,Erdem Tepegöz,"Jale Arıkan, Rüçhan Caliskur, Özay Fecht, Remz...",drama film,drama,https://en.wikipedia.org/wiki/Particle_(film),"Zeynep lost her job at weaving factory, and he...",zeynep lost job weaving factory family mother ...,job family mother daughter money try find new job,"(job, family, mother, daughter, money, try, fi...","[-0.1106521263718605, 0.1848858743906021, -0.2..."
42531,2017,Çalgı Çengi İkimiz,Turkish,Selçuk Aydemir,"Ahmet Kural, Murat Cemcir",comedy,comedy,https://en.wikipedia.org/wiki/%C3%87alg%C4%B1_...,"Two musicians, Salih and Gürkan, described the...",two musician salih gürkan described adventure ...,two,(two),"[0.1937599927186966, -0.34272000193595886, -0...."
42532,2017,Olanlar Oldu,Turkish,Hakan Algül,"Ata Demirer, Tuvana Türkay, Ülkü Duru",comedy,comedy,https://en.wikipedia.org/wiki/Olanlar_Oldu,"Zafer, a sailor living with his mother Döndü i...",zafer sailor living mother döndü coastal villa...,mother father also friend try help family find,"(mother, father, also, friend, try, help, fami...","[-0.0064917514100670815, 0.2386978566646576, -..."
42533,2017,Non-Transferable,Turkish,Brendan Bradley,"YouTubers Shanna Malcolm, Shira Lazar, Sara Fl...",romantic comedy,comedy,https://en.wikipedia.org/wiki/Non-Transferable...,The film centres around a young woman named Am...,film centre around young woman named amy tyler...,film young woman named however two go another ...,"(film, young, woman, named, however, two, go, ...","[0.04692091792821884, 0.1110081747174263, -0.1..."


In [10]:
len(Train['plot_sent_vectors'][7])

300

In [11]:
Train['plot_sent_vectors']

6        [-0.03173771873116493, -0.03446159139275551, -...
7        [0.06740987300872803, 0.1805768758058548, -0.1...
10       [0.10471398383378983, 0.09727063775062561, -0....
11       [0.1537410020828247, -0.14758041501045227, -0....
12       [0.0882359966635704, -0.030830401927232742, -0...
                               ...                        
42526    [-0.1106521263718605, 0.1848858743906021, -0.2...
42531    [0.1937599927186966, -0.34272000193595886, -0....
42532    [-0.0064917514100670815, 0.2386978566646576, -...
42533    [0.04692091792821884, 0.1110081747174263, -0.1...
42534    [0.11714199930429459, 0.2991870939731598, -0.2...
Name: plot_sent_vectors, Length: 35692, dtype: object

In [12]:
from sklearn.decomposition import PCA
sklearn_pca = PCA(n_components =2)
Y_sklearn = sklearn_pca.fit_transform(Train['plot_sent_vectors'].tolist())

In [13]:
cosine_sim= cosine_similarity(Y_sklearn ,Y_sklearn)

In [14]:
print(cosine_sim)

[[ 1.         -0.74704582  0.91018791 ... -0.80284224 -0.58253285
  -0.3047262 ]
 [-0.74704582  1.         -0.95529792 ...  0.20342819  0.97551062
  -0.40551153]
 [ 0.91018791 -0.95529792  1.         ... -0.48379742 -0.86687548
   0.11713832]
 ...
 [-0.80284224  0.20342819 -0.48379742 ...  1.         -0.01690676
   0.81248372]
 [-0.58253285  0.97551062 -0.86687548 ... -0.01690676  1.
  -0.59663703]
 [-0.3047262  -0.40551153  0.11713832 ...  0.81248372 -0.59663703
   1.        ]]


In [15]:
indices = pd.Series(Train.index, index=Train['Title']).drop_duplicates()
indices

Title
The Great Train Robbery                                  6
The Suburbanite                                          7
Dream of a Rarebit Fiend                                10
From Leadville to Aspen: A Hold-Up in the Rockies       11
From Leadville to Aspen: A Hold-Up in the Rockies       12
                                                     ...  
Particle (film)                                      42526
Çalgı Çengi İkimiz                                   42531
Olanlar Oldu                                         42532
Non-Transferable                                     42533
İstanbul Kırmızısı                                   42534
Length: 35692, dtype: int64

In [16]:
def get_recommendations(title, cosine_sim, indices):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:20]
    movie_indices = [i[0] for i in sim_scores]
    return set(Train['Title'].iloc[movie_indices])

In [19]:
recom=list(get_recommendations('The Suburbanite', cosine_sim, indices))
recom

['Run Chrissie Run!',
 'Captain Clegg',
 'Naked Ambition 2',
 'Medium Cool',
 'Ana Horra (I Am Free)',
 'Cairo Road',
 'Bronco Bullfrog',
 'Westward Ho']

In [23]:
metrics.average_precision.apk(list(indices.keys()),recom,10)

0.8

In [24]:
metrics.average_precision.mapk(list(indices.keys()),recom,10)

0.4278968253968254